In [ ]:
#! hitachicapitalvehiclesolutions ## 

In [28]:
import calendar
import time; 
  
# gmt stores current gmtime 
gmt = time.gmtime() 
print("gmt:-", gmt) 
  
# ts stores timestamp 
ts = calendar.timegm(gmt) 
print("timestamp:-", ts) 

gmt:- time.struct_time(tm_year=2021, tm_mon=2, tm_mday=2, tm_hour=14, tm_min=40, tm_sec=27, tm_wday=1, tm_yday=33, tm_isdst=0)
timestamp:- 1612276827


In [27]:
i=len("C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\1612771383\\HTML\\")
print(i,250-i)
string[0:160]

79 171


In [8]:
#!/usr/bin/env python
__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"
import calendar
import time; 
import os
gmt = time.gmtime() 
ts = calendar.timegm(gmt)
ts=str(ts)
#STEP1  WEB URL FUNCTIONs 
###Define Inputs###
#input#
brandingFile="c:\\hitachiPOC\\hitachicapitalvehiclesolutions\\input\\branding.txt"
domain="hitachicapitalvehiclesolutions"
parentlink="https://www.hitachicapitalvehiclesolutions.co.uk/" # give the Page URLs
userid=""
password=""
depth=5

#Define output folder
HTMLSTOREFOLDER="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\" + ts + "\\HTML\\"
RESULTFOLDER="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\" + ts + "\\RESULT\\"
exceptionFolder="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\" + ts + "\\exception\\"
outputFolder="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\" + ts + "\\output\\"

if not os.path.exists(HTMLSTOREFOLDER):
    os.makedirs(HTMLSTOREFOLDER)
if not os.path.exists(RESULTFOLDER):
    os.makedirs(RESULTFOLDER)
if not os.path.exists(exceptionFolder):
    os.makedirs(exceptionFolder)
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

URLFILEPATH_A =outputFolder +  "urls_full.txt" # Output URL Path
urlfilemappingFile=exceptionFolder+"URL_HTMLfile_mapping"+str(ts)+ ".txt"
urlfilemapping=open(urlfilemappingFile,"w")
exceptionFile=open(exceptionFolder+"generalException"+str(ts)+ ".txt","w")
exceptionHTTPFile=open(exceptionFolder+"HTTPexception.txt","w")
uniqURLS=open(outputFolder + "uniqURLS.txt","w")

urllinkmapping = RESULTFOLDER + "hitachicvs" + str(ts) + ".csv"

import calendar
import re
import sys
import time
import math
import urllib
import urllib.error
import urllib.request
from urllib.parse import urlparse
import optparse
import hashlib
#from cgi import escape
from html import escape
from traceback import format_exc
from queue import Queue, Empty as QueueEmpty
import csv
from bs4 import BeautifulSoup

__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"

USAGE = "%prog [options] <url>"
VERSION = "%prog v" + __version__

AGENT = "%s/%s" % (__name__, __version__)


#urlfilemappingFolder=open("c:\\hitachiPOC\\hitachicapitalvehiclesolutions\\exception\\hitachicvs-url_HTMLfile_mapping#14.txt","a")
username=""
password=""
class Link (object):

    def __init__(self, src, dst, link_type):
        self.src = src
        self.dst = dst
        self.link_type = link_type

    def __hash__(self):
        return hash((self.src, self.dst, self.link_type))

    def __eq__(self, other):
        return (self.src == other.src and
                self.dst == other.dst and
                self.link_type == other.link_type)
    
    def __str__(self):
        return self.src + " -> " + self.dst

class Crawler(object):

    def __init__(self, root, depth_limit, confine=None, exclude=[], locked=True, filter_seen=True):
        self.root = root
        self.host = urlparse(root)[1]

        ## Data for filters:
        self.depth_limit = depth_limit # Max depth (number of hops from root)
        self.locked = locked           # Limit search to a single host?
        self.confine_prefix=confine    # Limit search to this prefix
        self.exclude_prefixes=exclude; # URL prefixes NOT to visit
                

        self.urls_seen = set()          # Used to avoid putting duplicates in queue
        self.urls_remembered = set()    # For reporting to user
        self.visited_links= set()       # Used to avoid re-processing a page
        self.links_remembered = set()   # For reporting to user
        
        self.num_links = 0              # Links found (and not excluded by filters)
        self.num_followed = 0           # Links followed.  

        # Pre-visit filters:  Only visit a URL if it passes these tests
        self.pre_visit_filters=[self._prefix_ok,
                                self._exclude_ok,
                                self._not_visited,
                                self._same_host]

        # Out-url filters: When examining a visited page, only process
        # links where the target matches these filters.        
        if filter_seen:
            self.out_url_filters=[self._prefix_ok,
                                     self._same_host]
        else:
            self.out_url_filters=[]

    def _pre_visit_url_condense(self, url):
        
        """ Reduce (condense) URLs into some canonical form before
        visiting.  All occurrences of equivalent URLs are treated as
        identical.

        All this does is strip the \"fragment\" component from URLs,
        so that http://foo.com/blah.html\#baz becomes
        http://foo.com/blah.html """

        base, frag = urllib.parse.urldefrag(url)
        return base

    ## URL Filtering functions.  These all use information from the
    ## state of the Crawler to evaluate whether a given URL should be
    ## used in some context.  Return value of True indicates that the
    ## URL should be used.
    
    def _prefix_ok(self, url):
        """Pass if the URL has the correct prefix, or none is specified"""
        return (self.confine_prefix is None  or
                url.startswith(self.confine_prefix))

    def _exclude_ok(self, url):
        """Pass if the URL does not match any exclude patterns"""
        prefixes_ok = [ not url.startswith(p) for p in self.exclude_prefixes]
        return all(prefixes_ok)
    
    def _not_visited(self, url):
        """Pass if the URL has not already been visited"""
        return (url not in self.visited_links)
    
    def _same_host(self, url):
        """Pass if the URL is on the same host as the root URL"""
        try:
            host = urlparse(url)[1]
            return re.match(".*%s" % self.host, host) 
        except Exception as e:
            print(sys.stderr, "ERROR: Can't process url '%s' (%s)" % (url, e))
            return False
            

    def crawl(self):

        """ Main function in the crawling process.  Core algorithm is:

        q <- starting page
        while q not empty:
           url <- q.get()
           if url is new and suitable:
              page <- fetch(url)   
              q.put(urls found in page)
           else:
              nothing

        new and suitable means that we don't re-visit URLs we've seen
        already fetched, and user-supplied criteria like maximum
        search depth are checked. """
        
        q = Queue()
        q.put((self.root, 0))

        while not q.empty():
            this_url, depth = q.get()
            
            #Non-URL-specific filter: Discard anything over depth limit
            if depth > self.depth_limit:
                continue
            
            #Apply URL-based filters.
            do_not_follow = [f for f in self.pre_visit_filters if not f(this_url)]
            
            #Special-case depth 0 (starting URL)
            if depth == 0 and [] != do_not_follow:
                print(sys.stderr, "Whoops! Starting URL %s rejected by the following filters:", do_not_follow)

            #If no filters failed (that is, all passed), process URL
            if [] == do_not_follow:
                try:
                    self.visited_links.add(this_url)
                    self.num_followed += 1
                    page = Fetcher(this_url)
                    page.fetch()
                    for link_url in [self._pre_visit_url_condense(l) for l in page.out_links()]:
                        if link_url not in self.urls_seen:
                            q.put((link_url, depth+1))
                            self.urls_seen.add(link_url)
                            
                        do_not_remember = [f for f in self.out_url_filters if not f(link_url)]
                        if [] == do_not_remember:
                                self.num_links += 1
                                self.urls_remembered.add(link_url)
                                link = Link(this_url, link_url, "href")
                                if link not in self.links_remembered:
                                    self.links_remembered.add(link)
                except Exception as e:
                    print(sys.stderr, "ERROR: Can't process url '%s' (%s)" % (this_url, e))
                    #print format_exc()

class OpaqueDataException (Exception):
    def __init__(self, message, mimetype, url):
        Exception.__init__(self, message)
        self.mimetype=mimetype
        self.url=url
        

class Fetcher(object):
    
    """The name Fetcher is a slight misnomer: This class retrieves and interprets web pages."""

    def __init__(self, url):
        self.url = url
        self.out_urls = []

    def __getitem__(self, x):
        return self.out_urls[x]

    def out_links(self):
        return self.out_urls

    def _addHeaders(self, request):
        request.add_header("User-Agent", AGENT)

    def _open(self):
        url = self.url
        try:
            request = urllib.request.Request(url)
            handle = urllib.request.build_opener()
        except IOError:
            return None
        return (request, handle)

    def fetch(self):
        request, handle = self._open()
        self._addHeaders(request)
        if handle:
            try:
                data=handle.open(request)
                mime_type=data.info().get_content_type()
                url=data.geturl();
                if mime_type != "text/html":
                    raise OpaqueDataException("Not interested in files of type %s" % mime_type,
                                              mime_type, url)
                aFile=re.sub('\\n|\\r','',url)
                aFile=re.sub("[./=?:]","_",aFile)
                if (len(aFile) > 160):
                    aFile=aFile[0:160]
                
                fOutFileName=HTMLSTOREFOLDER+aFile+".html"
                                   
                content = data.read().decode("utf-8", errors="replace")
                
                try:
                    with open(fOutFileName,"w", encoding='utf-8') as outFile:
                        outFile.write(str(content))
                    #print("fetch Created File:[%s]" % (str(len(aFile))))
                    print(url+"::"+fOutFileName,file=urlfilemapping)
                    #urlfilemapping.close()
                    soup = BeautifulSoup(content, "html.parser")
                    tags = soup('a')
                except Exception as e:
                    print("Cannot Open File",e,url,len(aFile),fOutFileName)
                    print("Cannot Open File:[%s]" % (fOutFileName))
                    print("Cannot Open File::"+fOutFileName,file=exceptionFile)
                    return 80
            except urllib.error.HTTPError as error:
                if error.code == 404:
                    print(sys.stderr, "HTTP ERROR: %s -> %s" % (error, error.url))
                    print(error+"::"+error.url,file=exceptionFile)
                else:
                    print(sys.stderr, "HTTP ERROR: %s" % error+error.url)
                    print(error+"::"+error.url,file=exceptionFile)
                tags = []
            except urllib.error.URLError as error:
                print(sys.stderr, "URL ERROR: %s" % error)
                tags = []
            except OpaqueDataException as error:
                print(sys.stderr, "Skipping %s, has type %s" % (error.url, error.mimetype))
                tags = []
            for tag in tags:
                href = tag.get("href")
                if href is not None:
                    url = urllib.parse.urljoin(self.url, escape(href))
                    if url not in self:
                        self.out_urls.append(url)

def getLinks(url,outMappingfile,URLFILEINP):
  
    try:
        page = Fetcher(url)
        page.fetch()
    except Exception as e:
        print("Error in getLinks URL",url,e)
        
    Hitachi_gURL_link_Mapping_File = open(outMappingfile, 'a', newline='')
    Hitachi_gURL_link_Mapping_File_writer = csv.writer(Hitachi_gURL_link_Mapping_File)
    #Hitachi_gURL_fURL_Mapping_File_File_writer.writerow(["gURL","fURL"])
    Row_data = []

    URLFILE=open(URLFILEINP,"a")
    
    for i, urllink in enumerate(page):
        try:
            #print(":::::::",urllink)
            print(urllink,file=URLFILE)
            Row_data.clear()
            Row_data.append(url)
            Row_data.append(urllink)
            Hitachi_gURL_link_Mapping_File_writer.writerow(Row_data)
        except UnicodeDecodeError:
            print("UnicodeDecodeError:",url)
            #print("%d. %s" % (i, url))
    Hitachi_gURL_link_Mapping_File.close()
    
def parse_options():
    """parse_options() -> opts, args

    Parse any command-line options given returning both
    the parsed options and arguments.
    """

    parser = optparse.OptionParser(usage=USAGE, version=VERSION)

    parser.add_option("-q", "--quiet",
            action="store_true", default=False, dest="quiet",
            help="Enable quiet mode")

    parser.add_option("-l", "--links",
            action="store_true", default=False, dest="links",
            help="Get links for specified url only")    

    parser.add_option("-d", "--depth",
            action="store", type="int", default=30, dest="depth_limit",
            help="Maximum depth to traverse")

    parser.add_option("-c", "--confine",
            action="store", type="string", dest="confine",
            help="Confine crawl to specified prefix")

    parser.add_option("-x", "--exclude", action="append", type="string",
                      dest="exclude", default=[], help="Exclude URLs by prefix")
    
    parser.add_option("-L", "--show-links", action="store_true", default=False,
                      dest="out_links", help="Output links found")

    parser.add_option("-u", "--show-urls", action="store_true", default=False,
                      dest="out_urls", help="Output URLs found")

    parser.add_option("-D", "--dot", action="store_true", default=False,
                      dest="out_dot", help="Output Graphviz dot file")
    


    opts, args = parser.parse_args()

    if len(args) < 1:
        parser.print_help(sys.stderr)
        raise SystemExit(1)

    if opts.out_links and opts.out_urls:
        parser.print_help(sys.stderr)
        parser.error("options -L and -u are mutually exclusive")

    return opts, args

class DotWriter:

    """ Formats a collection of Link objects as a Graphviz (Dot)
    graph.  Mostly, this means creating a node for each URL with a
    name which Graphviz will accept, and declaring links between those
    nodes."""

    def __init__ (self):
        self.node_alias = {}

    def _safe_alias(self, url, silent=False):

        """Translate URLs into unique strings guaranteed to be safe as
        node names in the Graphviz language.  Currently, that's based
        on the md5 digest, in hexadecimal."""

        if url in self.node_alias:
            return self.node_alias[url]
        else:
            m = hashlib.md5()
            m.update(url)
            name = "N"+m.hexdigest()
            self.node_alias[url]=name
            if not silent:
                print("\t%s [label=\"%s\"];" % (name, url))
            return name


    def asDot(self, links):

        """ Render a collection of Link objects as a Dot graph"""
        
        print("digraph Crawl {")
        print("\t edge [K=0.2, len=0.1];")
        for l in links:            
            print("\t" + self._safe_alias(l.src) + " -> " + self._safe_alias(l.dst) + ";")
        print("}")

        
    

def main():    
    opts, args = parse_options()

    url = args[0]

    if opts.links:
        getLinks(url)
        raise SystemExit(0)

    depth_limit = opts.depth_limit
    confine_prefix=opts.confine
    exclude=opts.exclude

    sTime = time.time()

    print(sys.stderr,  "Crawling %s (Max Depth: %d)" % (url, depth_limit))
    crawler = Crawler(url, depth_limit, confine_prefix, exclude)
    crawler.crawl()

    if opts.out_urls:
        print("\n".join(crawler.urls_seen))

    if opts.out_links:
        print("\n".join([str(l) for l in crawler.links_remembered]))
        
    if opts.out_dot:
        d.asDot(crawler.links_remembered)

    eTime = time.time()
    tTime = eTime - sTime

    print(sys.stderr, "Found:    %d" % crawler.num_links)
    print(sys.stderr, "Followed: %d" % crawler.num_followed)
    print(sys.stderr, "Stats:    (%d/s after %0.2fs)" % (int(math.ceil(float(crawler.num_links) / tTime)), tTime))

In [9]:
print("######--------------------------------------------------------------#######")
print("HTMLSTOREFOLDER-",HTMLSTOREFOLDER)
print("RESULTFOLDER-",RESULTFOLDER)
print("exceptionFolder-",exceptionFolder)
print("outputFolder-",outputFolder)
print("URLFILEPATH_A-",URLFILEPATH_A)
print("urlfilemapping-",urlfilemapping)
print("exceptionFile-",exceptionFile)
print("exceptionHTTPFile-",exceptionHTTPFile)
print("uniqURLS-",uniqURLS)
print("DEPTH-",depth)
print("brandingFile-",brandingFile)
print("parentlink-",parentlink)
print("urllinkmapping-",urllinkmapping)
print("######--------------------------------------------------------------#######")

######--------------------------------------------------------------#######
HTMLSTOREFOLDER- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\HTML\
RESULTFOLDER- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\RESULT\
exceptionFolder- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\exception\
outputFolder- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\output\
URLFILEPATH_A- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\output\urls_full.txt
urlfilemapping- <_io.TextIOWrapper name='C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\1612880500\\exception\\URL_HTMLfile_mapping1612880500.txt' mode='w' encoding='cp1252'>
exceptionFile- <_io.TextIOWrapper name='C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\1612880500\\exception\\generalException1612880500.txt' mode='w' encoding='cp1252'>
exceptionHTTPFile- <_io.TextIOWrapper 

In [10]:
#Step 2 URL Domain Setting parentlink , uniqURLS - DownLoad Webpages 
__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"

print("######--------------------------------------------------------------#######")
print("HTMLSTOREFOLDER-",HTMLSTOREFOLDER)
print("RESULTFOLDER-",RESULTFOLDER)
print("exceptionFolder-",exceptionFolder)
print("outputFolder-",outputFolder)
print("URLFILEPATH_A-",URLFILEPATH_A)
print("urlfilemapping-",urlfilemapping)
print("exceptionFile-",exceptionFile)
print("exceptionHTTPFile-",exceptionHTTPFile)
print("uniqURLS-",uniqURLS)
print("DEPTH-",depth)
print("brandingFile-",brandingFile)
print("parentlink-",parentlink)
print("urllinkmapping-",urllinkmapping)
print("######--------------------------------------------------------------#######")
import calendar 
import datetime
ts = calendar.timegm(gmt)
getLinks(parentlink,urllinkmapping,URLFILEPATH_A)
processedURL = []
print("startTime",datetime.datetime.now())
finalurl=""
for x in range(depth):
    data = []
    data = [line.strip() for line in open(URLFILEPATH_A, 'r')]
    #print(x)
    for eachurl in data:
        finalurl=eachurl
        #print(eachurl)
        if "javascript:;" in eachurl.lower():
            #print(eachurl,file=exceptionSpecialCharFile)
            continue
        if ".pdf" in eachurl.lower():
            #print(eachurl,file=pdfurlFile)
            continue
        if "facebook." in eachurl.lower():
            #print(eachurl,file=pdfurlFile)
            continue
        if "twitter." in eachurl.lower(): #linkedin
            #print(eachurl,file=pdfurlFile)
            continue
        if "linkedin." in eachurl.lower(): #linkedin
            #print(eachurl,file=pdfurlFile)
            continue
        if "google." in eachurl.lower(): #linkedin
            #print(eachurl,file=pdfurlFile)
            continue
        if ".css" in  eachurl.lower() or ".ico" in  eachurl.lower() :
            continue
        if  "mailto:" in  eachurl.lower():
            continue
        if  "tel:" in  eachurl.lower() :
            continue
        if  ".gif" in  eachurl.lower() or ".png" in  eachurl.lower() :
            continue
        if  ".jpg" in eachurl.lower() or ".xml" in  eachurl.lower() :
            continue
        if domain in eachurl.lower() or  "hcvs" in eachurl.lower() or "hpf" in eachurl.lower():
            if eachurl in processedURL:
                continue;
            eachurl=str(eachurl)
            #print("::",eachurl)
            try:
                getLinks(eachurl.replace(" ","%20"),urllinkmapping,URLFILEPATH_A)
                processedURL.append(eachurl)
            except Exception as e :
                print(eachurl,file=exceptionHTTPFile)
                print("Problem URL:" , eachurl , e)
for uurl in processedURL:
    print(uurl,file=uniqURLS)
print("Final endTime",datetime.datetime.now())
urlfilemapping.close()
#URLFILEPATH_A.close()
exceptionFile.close()
exceptionHTTPFile.close()
uniqURLS.close()

######--------------------------------------------------------------#######
HTMLSTOREFOLDER- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\HTML\
RESULTFOLDER- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\RESULT\
exceptionFolder- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\exception\
outputFolder- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\output\
URLFILEPATH_A- C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\output\urls_full.txt
urlfilemapping- <_io.TextIOWrapper name='C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\1612880500\\exception\\URL_HTMLfile_mapping1612880500.txt' mode='w' encoding='cp1252'>
exceptionFile- <_io.TextIOWrapper name='C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\1612880500\\exception\\generalException1612880500.txt' mode='w' encoding='cp1252'>
exceptionHTTPFile- <_io.TextIOWrapper 

In [11]:
#Step 3 Create URL Classification
__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"
print("startTime",datetime.datetime.now())
import csv
import time
with open(URLFILEPATH_A) as file:
    uniquerulslines = [line.strip() for line in file]
#print(uniquerulslines)
donmaiurlsonly=open(RESULTFOLDER + "domain-links#5.txt","a")
Hitachi_Link_Classifcation_File = open(RESULTFOLDER + 'Link_Classifcation_File_Analysis_Run#3.csv', 'w', newline='')
Hitachi_Link_Classifcation_File_writer = csv.writer(Hitachi_Link_Classifcation_File)
Hitachi_Link_Classifcation_File_writer.writerow(["Link Type","Full Link"])
Row_data = []
processedURLVS = []
for line in uniquerulslines:
    if line in processedURLVS:
        continue;
    Row_data.clear()
    #print(line)
    if ".pdf" in line.lower() and domain in line.lower():
        Row_data.append(".pdf-" + domain )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if ".pdf" in line.lower() :
        Row_data.append(".pdf" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    #--
    
    if "facebook." in line.lower() and domain in line.lower():
        Row_data.append("facebook-"+domain )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if "twitter." in line.lower() and domain in line.lower():
        Row_data.append("twitter-"+domain ) 
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if "linkedin." in line.lower() and domain in line.lower():
        Row_data.append("linkedin-"+domain )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if ".html" in line.lower() and domain in line.lower():
        Row_data.append(".html-"+domain )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if ".css" in line or ".ico" in line.lower() and domain in line.lower():
        Row_data.append("css or ico-"+domain )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if  "mailto:" in line.lower() and domain in line.lower():
        Row_data.append("mailto:-" + domain )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if  "Tel:" in line or "tel:" in line.lower() and domain in line.lower():
        Row_data.append("Tel-" + domain )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    
    #--
        
    if "facebook." in line.lower():
        Row_data.append("facebook" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if "twitter." in line.lower():
        Row_data.append("twitter" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if "linkedin." in line.lower():
        Row_data.append("linkedin" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if ".html" in line.lower():
        Row_data.append(".html" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if ".css" in line or ".ico" in line.lower() :
        Row_data.append("css or ico" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if  "mailto:" in line.lower():
        Row_data.append("mailto:" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if  "Tel:" in line or "tel:" in line.lower() :
        Row_data.append("Tel" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if line.startswith("/") or line.startswith("#")  :
        #print("Hitachi-Relative-Link",line)
        Row_data.append("Hitachi-Relative-Link" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    if  domain in line.lower() :
        Row_data.append(domain+"-Link" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        print(line,file=donmaiurlsonly)
        continue
    if  "hitachi" in line.lower() :
        Row_data.append("Non-" + domain + "-Link" )
        Row_data.append(line )
        Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
        processedURLVS.append(line)
        continue
    #print("Non-Hitachi-Link",line)
    Row_data.append("Non-Hitachi-Link" )
    Row_data.append(line )
    Hitachi_Link_Classifcation_File_writer.writerow(Row_data)
    processedURLVS.append(line)
Hitachi_Link_Classifcation_File.close()
print("URL Classification Completed OutputFile:" , RESULTFOLDER + 'Link_Classifcation_File_Analysis_Run#3.csv')
print("endTime",datetime.datetime.now())

startTime 2021-02-09 23:20:03.634363
URL Classification Completed OutputFile: C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\RESULT\Link_Classifcation_File_Analysis_Run#3.csv
endTime 2021-02-09 23:20:03.669268


In [18]:
#Find Branding instances in HTML
__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"
import os
#HTMLSTOREFOLDER="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\1612776574\\HTML\\"
#outputFolder="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\URLBradingAnaysis\\1612776574\\output\\"
#brandingFile="c:\\hitachiPOC\\hitachicapitalvehiclesolutions\\input\\branding.txt"
#arr = os.listdir(HTMLSTOREFOLDER)
#print(arr)


with open(urlfilemappingFile,"r") as URLMAP:
    urlFileMapping = [line.strip() for line in URLMAP]
URLMAP.close() 
#print(urlFileMapping)
#
with open(brandingFile) as BRAND:
    brandings = [line.strip() for line in BRAND]
BRAND.close() 

Hitachi_BRAND_File = open(outputFolder + 'BRANDING_Analysis_Run#3.csv', 'w', newline='')
Hitachi_BRAND_File_writer = csv.writer(Hitachi_BRAND_File)
Hitachi_BRAND_File_writer.writerow([ "URL" , "BRAND INSTANCE FOUND" ,"COUNT"])

Hitachi_DETAILED_BRAND_File = open(outputFolder + 'BRANDING_DETAILED_Analysis_Run#3.csv', 'w', newline='',encoding='utf-8')
Hitachi_DETAILED_BRAND_File_writer = csv.writer(Hitachi_DETAILED_BRAND_File)
Hitachi_DETAILED_BRAND_File_writer.writerow([ "URL" ,"BRAND INSTANCE FOUND" ,"Relevant snippets / Context"])

Row_data = []
DETAILED_BRAND_Row_data = []

for eachfile in urlFileMapping:
    i=eachfile.index("::")
    end=len(eachfile)
    url=eachfile[0:i]
    aFile =  eachfile[i+2:end]
    with open(aFile,"r",encoding='utf-8') as INPFILE:
        lFileContents=INPFILE.read()
        soup = BeautifulSoup(lFileContents, 'html.parser')
    INPFILE.close()
    brandsString=""
    count=0
    findStringInstanceList=[]
    brandCountDict={}
    for aBarndItem in brandings:
        findStringInstanceList=[]
        findStringInstanceList=soup.findAll(text = re.compile(aBarndItem))
        eachBarndCount =0
        if (len(findStringInstanceList) > 0) : 
            brandCountDict[aBarndItem] = len(findStringInstanceList)
            count = count +1
        for eachInstance in findStringInstanceList :
            DETAILED_BRAND_Row_data.clear()
            DETAILED_BRAND_Row_data.append(url)
            DETAILED_BRAND_Row_data.append(aBarndItem)
            if (len(eachInstance) > 100):
                start = str(eachInstance).index(aBarndItem)
                end = str(eachInstance).index(aBarndItem) + 100
                eachInstance = eachInstance[start:end]
            #eachInstance.index("aBarndItem")
            DETAILED_BRAND_Row_data.append(eachInstance)
            Hitachi_DETAILED_BRAND_File_writer.writerow(DETAILED_BRAND_Row_data)
    if (count > 0) :       
        Row_data.clear()
        #Row_data.append(aFile )
        finalCount=""
        finalBrandString=""
        for eachinstancecount in brandCountDict:
            finalBrandString = finalBrandString + eachinstancecount + ";"
            finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
        Row_data.append(url)
        Row_data.append(finalBrandString)
        Row_data.append(finalCount)
        Hitachi_BRAND_File_writer.writerow(Row_data)
        #print(brandsString + "found in" +  aFile + "count" + str(count)  )  
Hitachi_BRAND_File.close()
Hitachi_DETAILED_BRAND_File.close()
print("Find Branding instances in URL Completed",outputFolder + 'BRANDING_Analysis_Run#3.csv',outputFolder + 'BRANDING_DETAILED_Analysis_Run#3.csv')

Find Branding instances in URL Completed C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\output\BRANDING_Analysis_Run#3.csv C:\hitachiPOC\hitachicapitalvehiclesolutions\URLBradingAnaysis\1612880500\output\BRANDING_DETAILED_Analysis_Run#3.csv


In [ ]:
#Source CODE Branding Analysis#

In [25]:
import fitz  # this is pymupdf
import os
from tika import parser
import re

# import re
# xx = "guru99,education is fun"
# r1 = re.findall(r"^\w+",xx)
# print(r1)

def _PDF (fname,pdfpath,brandings):
    fileextract=""
    
    returnDict={}
    brandCountDict={}
    try:
        raw = parser.from_file(pdfpath+fname)
        string = str(raw['content'].encode('ascii', errors='ignore'))
        count=0
        brandsString=""
        for aBarndItem in brandings:
            find_the_word = re.finditer(aBarndItem, string)
            snippetList =[]
            for match in find_the_word:
                start=match.start()-50
                end=match.end()+50
                if (start  < 0 ):
                    start = 0
                if (len(string) < end):
                    end = len(string)
                finalstring = string[start:end].replace("\\n", " ")
                if len(finalstring) > 250:
                    finalstring = finalstring[0:150]
                snippetList.append(finalstring)
            if (len(snippetList) > 0):
                brandCountDict[aBarndItem] = len(snippetList)
                returnDict[aBarndItem] = snippetList
        #print(raw['content'])
    except Exception as e:
        print("problem in PDF Parsing" + pdfpath + fname , e)
   
    return (brandCountDict,returnDict)

def _XLS (fname,xlpath,brandings):
    brandsString=""
    count=0
    returnDict={}
    brandCountDict={}
    try:
        from xlrd import open_workbook
        book = open_workbook(xlpath+fname)
        p = book.sheet_names()
        sheet_data=[]
        for y in p:
            sh = book.sheet_by_name(y)
            for rownum in range(sh.nrows):
                sheet_data.append((sh.row_values(rownum)))
        for i in sheet_data:
            find_the_words = []
            for aBarndItem in brandings:
                find_the_word = re.finditer(aBarndItem, i[2])
                snippetList =[]
                
                for match in find_the_word:
                    start=match.start()-50
                    end=match.end()+50
                    if (start  < 0 ):
                        start = 0
                    if (len(string) < end):
                        end = len(string)
                    finalstring = string[start:end].replace("\\n", " ")
                    if len(finalstring) > 250:
                        finalstring = finalstring[0:150]
                    snippetList.append(finalstring)
                if (len(snippetList) > 0):
                    brandCountDict[aBarndItem] = len(snippetList)
                    returnDict[aBarndItem] = snippetList   
    except Exception as e:
        print("problem in XL File" + xlpath + fname ,e)
    return (brandCountDict,returnDict)

def _DOC (fname,docpath,brandings):
    
   
    brandsString=""
    returnDict={}
    try:
        raw = parser.from_file(docpath+fname)
        string = str(raw['content'].encode('ascii', errors='ignore'))
        brandCountDict={}
        returnDict={}
        for aBarndItem in brandings:
            find_the_word = re.finditer(aBarndItem, string)
            snippetList =[]
            for match in find_the_word:
                start=match.start()-50
                end=match.end()+50
                if (start  < 0 ):
                    start = 0
                if (len(string) < end):
                    end = len(string)
                finalstring = string[start:end].replace("\\n", " ")
                if len(finalstring) > 250:
                    finalstring = finalstring[0:150]
                snippetList.append(finalstring)
            
            #snippetList = [x.group() for x in haRegex.search(string)]
            #sentences = string.split(".") 
            #print(snippetList)
            if (len(snippetList) > 0):
                #count = count + 1
                brandCountDict[aBarndItem] = len(snippetList)
                returnDict[aBarndItem] = snippetList
        #print(raw['content'])
    except Exception as e:
        print("problem in MS Doc Parsing" + docpath + fname , e)
   
    return (brandCountDict,returnDict)

def _XML (fname,xmlpath,brandings):
    #print("_XML")
    brandsString=""
    count=0
    returnDict={}
    brandCountDict={}
    try:
        with open(xmlpath+fname,"r") as INPFILE:
            lFileContents=INPFILE.read()
            #print(lFileContents)
            soup = BeautifulSoup(lFileContents, 'xml')
        INPFILE.close()
       
        for aBarndItem in brandings:
            findStringInstanceList=soup.findAll(text = re.compile(aBarndItem))
            snippetList =[]
            
            for eachInstance in findStringInstanceList :
                if (len(eachInstance) > 100):
                    start = str(eachInstance).index(aBarndItem)
                    end = str(eachInstance).index(aBarndItem) + 100
                    eachInstance = eachInstance[start:end]
                snippetList.append(eachInstance)
            if (len(snippetList) > 0):
                brandCountDict[aBarndItem] = len(snippetList)
                returnDict[aBarndItem] = snippetList
    except Exception as e:
        print("Warning problem in XML or RDF " + xmlpath + fname ,e)
    return (brandCountDict,returnDict)

def _HTML (fname,xmlpath,brandings):
    returnDict={}
    brandCountDict={}
    try:
        with open(xmlpath+fname,"r") as INPFILE:
            lFileContents=INPFILE.read()
            soup = BeautifulSoup(lFileContents, 'html.parser')
        INPFILE.close()
        brandsString=""
        count=0
        for aBarndItem in brandings:
            findStringInstanceList=soup.findAll(text = re.compile(aBarndItem))
            snippetList =[]
            for eachInstance in findStringInstanceList :
                if (len(eachInstance) > 100):
                    start = str(eachInstance).index(aBarndItem)
                    end = str(eachInstance).index(aBarndItem) + 100
                    eachInstance = eachInstance[start:end]
                snippetList.append(finalstring)
            if (len(snippetList) > 0):
                brandCountDict[aBarndItem] = len(snippetList)
                returnDict[aBarndItem] = snippetList
    except Exception as e:
        print("Warning problem in HTML" + xmlpath + fname ,e)
    return (brandCountDict,returnDict)

def _RDF (fname,xmlpath,brandings):
    #print("_RDF")
    brandsString=""
    count=0
    str1 = " " 
    returnDict={}
    try:
        with open(xmlpath+fname,"r") as INPFILE:
            lFileContents=INPFILE.read()
        INPFILE.close()
        #print("_RDF"+lFileContents)
        import rdflib
        graph = rdflib.Graph()
        graph.parse(data=lFileContents)
        outputRDP = []
        for s, p, o in graph:
            if type(o) == rdflib.term.Literal:
                outputRDP.append(o.toPython())
        #print(outputRDP)
        for aBarndItem in brandings:
            snippetList =[]
            for eachrdp in outputRDP:
                if aBarndItem.lower() in eachrdp.lower():
                    snippetList.append(eachrdp)
            if (len(snippetList) > 0):
                brandCountDict[aBarndItem] = len(snippetList)
                returnDict[aBarndItem] = snippetList
    except Exception as e:
        print("Warning problem in RDF " + xmlpath + fname ,e)
    return (brandCountDict,returnDict)

def findMailMERGETemplate (fname,docspath,brandings):
    from mailmerge import MailMerge
    t=0
    try:
        with MailMerge(docspath+fname) as document:
             t = len(document.get_merge_fields())
        document.close()
    except Exception as e:
        print("Warning problem in DOCS " + docspath + fname ,e)
    return(t)

In [29]:
#Source CODE Branding Analysis
#Source CODE Branding Analysis
#Source CODE Branding Analysis
import calendar
import time; 
import os;
import csv
from bs4 import BeautifulSoup
__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"
gmt = time.gmtime() 
ts = calendar.timegm(gmt)
ts=str(ts)
CODEPARENTFOLDER="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\CODEBASE\\"
CodeoutputFolder="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\CODEBradingAnaysis\\" + ts + "\\output\\"
brandingFile="c:\\hitachiPOC\\hitachicapitalvehiclesolutions\\input\\branding.txt"
exceptionFolder="C:\\hitachiPOC\\hitachicapitalvehiclesolutions\\CODEBradingAnaysis\\" + ts + "\\exception\\"

if not os.path.exists(CodeoutputFolder):
    os.makedirs(CodeoutputFolder)
if not os.path.exists(exceptionFolder):
    os.makedirs(exceptionFolder)
expfile=exceptionFolder+"generalExceptionNotprocessed"+ts+".txt"
exceptionFile=open(expfile,"w")

print("list of not process",file=exceptionFile)   
from os import listdir
from os.path import isfile, join
import os
subdirs = [x[0] for x in os.walk(CODEPARENTFOLDER)]
#print(subdirs)

CODEBRAND_File = CodeoutputFolder + 'CODE_BRANDING_Analysis_Run#66.csv'
DETAILED_CODEBRAND_File = CodeoutputFolder + 'CODE_Detailed_BRANDING_Analysis_Run#66.csv'

Hitachi_CODEBRAND_File = open(CODEBRAND_File, 'w', newline='')
Hitachi_CODEBRAND_File_writer = csv.writer(Hitachi_CODEBRAND_File)
Hitachi_CODEBRAND_File_writer.writerow(["Source Code File Path" , "TYPE" , "BRAND INSTANCE FOUND","COUNT"])

Hitachi_DETAILED_CODEBRAND_File = open(DETAILED_CODEBRAND_File, 'w', newline='')
Hitachi_DETAILED_CODEBRAND_File_writer = csv.writer(Hitachi_DETAILED_CODEBRAND_File)
Hitachi_DETAILED_CODEBRAND_File_writer.writerow(["Source Code File Path" , "TYPE" , "BRAND INSTANCE FOUND","Relevant snippets / Context"])

with open(brandingFile) as BRAND:
    brandings = [line.strip() for line in BRAND]
BRAND.close() 
Row_data = []
Row_DETAILED_data = []
fileCount=0
for eachfolder in subdirs:
    #print("mainloop---------"+eachfolder)
    onlyfiles = [f for f in listdir(eachfolder) if isfile(join(eachfolder, f))]
    eachfolder = eachfolder + "\\" 
    for eachfile in onlyfiles:
        fileCount=fileCount+1
         #print("Processing..." + eachfolder+eachfile)
        lFileContents=[]
        if ("~$S" in eachfile):
            print("Skipping Lock Files:" + eachfile )
            continue
        
        if (".html" in eachfile):
            brandsString=""
            brandCountDict,returnDict= _HTML(eachfile,eachfolder,brandings)
            if (len(brandCountDict) > 0):
                Row_data.clear()
                finalCount=""
                finalBrandString=""
                for eachinstancecount in brandCountDict:	
                    finalBrandString = finalBrandString + eachinstancecount + ";"	
                    finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                Row_data.append(eachfolder+eachfile )
                Row_data.append("HTML")
                Row_data.append(finalBrandString)
                Row_data.append(finalCount)
                #print("processed:"+,file=exceptionFile) 
                #print(eachfolder+eachfile,brandsString,count)
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            else:
                Row_data.clear()
                Row_data.append(eachfolder+eachfile )
                Row_data.append("HTML")
                Row_data.append("NONE")
                Row_data.append("NA")
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            if len(returnDict) > 0 :
                for eachinstance in returnDict:
                    #list = []
                    list = returnDict[eachinstance]
                    #print(":::list",list)
                    for l in list:
                        #print("inside" , l)
                        Row_DETAILED_data.clear()
                        Row_DETAILED_data.append(eachfolder+eachfile )
                        Row_DETAILED_data.append("HTML")
                        Row_DETAILED_data.append(eachinstance)
                        Row_DETAILED_data.append(l)
                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
            continue
        if (".xml" in eachfile):
            brandsString=""
            brandCountDict,returnDict= _XML(eachfile,eachfolder,brandings)
            if (len(brandCountDict) > 0):
                Row_data.clear()
                finalCount=""
                finalBrandString=""
                for eachinstancecount in brandCountDict:	
                    finalBrandString = finalBrandString + eachinstancecount + ";"	
                    finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                Row_data.append(eachfolder+eachfile )
                Row_data.append("XML")
                Row_data.append(finalBrandString)
                Row_data.append(finalCount)
                #print(eachfolder+eachfile,brandsString,count)
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            else:
                Row_data.clear()
                Row_data.append(eachfolder+eachfile )
                Row_data.append("XML")
                Row_data.append("NONE")
                Row_data.append("NA")
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            if len(returnDict) > 0 :
                for eachinstance in returnDict:
                    #list = []
                    list = returnDict[eachinstance]
                    #print(":::list",list)
                    for l in list:
                        #print("inside" , l)
                        Row_DETAILED_data.clear()
                        Row_DETAILED_data.append(eachfolder+eachfile )
                        Row_DETAILED_data.append("XML")
                        Row_DETAILED_data.append(eachinstance)
                        Row_DETAILED_data.append(l)
                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
            continue
        if (".rdf" in eachfile):
            #continue
            brandsString=""
            brandCountDict,returnDict= _RDF(eachfile,eachfolder,brandings)
            if (len(brandCountDict) > 0):
                Row_data.clear()
                finalCount=""
                finalBrandString=""
                for eachinstancecount in brandCountDict:	
                    finalBrandString = finalBrandString + eachinstancecount + ";"	
                    finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                Row_data.append(eachfolder+eachfile )
                Row_data.append("RDF")
                Row_data.append(finalBrandString)
                Row_data.append(finalCount)
                #print(eachfolder+eachfile,brandsString,count)
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            else:
                Row_data.clear()
                Row_data.append(eachfolder+eachfile )
                Row_data.append("NONE")
                Row_data.append("RDF")
                Row_data.append(count)
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            if len(returnDict) > 0 :
                for eachinstance in returnDict:
                    #list = []
                    list = returnDict[eachinstance]
                    #print(":::list",list)
                    for l in list:
                        #print("inside" , l)
                        Row_DETAILED_data.clear()
                        Row_DETAILED_data.append(eachfolder+eachfile )
                        Row_DETAILED_data.append("RDF")
                        Row_DETAILED_data.append(eachinstance)
                        Row_DETAILED_data.append(l)
                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
            continue
        if (".pdf" in eachfile):
            brandsString=""
            brandCountDict,returnDict= _PDF(eachfile,eachfolder,brandings)
            if (len(brandCountDict) > 0):
                Row_data.clear()
                finalCount=""
                finalBrandString=""
                for eachinstancecount in brandCountDict:	
                    finalBrandString = finalBrandString + eachinstancecount + ";"	
                    finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                Row_data.append(eachfolder+eachfile )
                Row_data.append("PDF")
                Row_data.append(finalBrandString)
                Row_data.append(finalCount)
                #print(eachfolder+eachfile,brandsString,count)
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            else:
                Row_data.clear()
                Row_data.append(eachfolder+eachfile )
                Row_data.append("PDF")
                Row_data.append("NONE")
                Row_data.append("NA")
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            #print("==pdf",len(returnDict))
            if len(returnDict) > 0 :
                for eachinstance in returnDict:
                    list = returnDict[eachinstance]
                    #print(":::list",list)
                    for l in list:
                        #print("inside" , l)
                        Row_DETAILED_data.clear()
                        Row_DETAILED_data.append(eachfolder+eachfile )
                        Row_DETAILED_data.append("PDF")
                        Row_DETAILED_data.append(eachinstance)
                        Row_DETAILED_data.append(l)
                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                        #print(eachfolder+eachfile , "PDF" ,  eachinstance ,  l)
            continue
        if (".xlsx" in eachfile or ".xls" in eachfile):
            
            brandCountDict,returnDict= _XLS(eachfile,eachfolder,brandings)
            if (len(brandCountDict) > 0):
                Row_data.clear()
                finalCount=""
                finalBrandString=""
                for eachinstancecount in brandCountDict:
                    finalBrandString = finalBrandString + eachinstancecount + ";"
                    finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                Row_data.append(eachfolder+eachfile )
                Row_data.append("XL-Sheet")
                Row_data.append(finalBrandString)
                Row_data.append(finalCount)
                #print(eachfolder+eachfile,brandsString,count)
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            else:
                Row_data.clear()
                Row_data.append(eachfolder+eachfile )
                Row_data.append("XL-Sheet")
                Row_data.append("NONE")
                Row_data.append("NA")
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            if len(returnDict) > 0 :
                for eachinstance in returnDict:
                    #list = []
                    list = returnDict[eachinstance]
                    #print(":::list",list)
                    for l in list:
                        #print("inside" , l)
                        Row_DETAILED_data.clear()
                        Row_DETAILED_data.append(eachfolder+eachfile )
                        Row_DETAILED_data.append("XL-Sheet")
                        Row_DETAILED_data.append(eachinstance)
                        Row_DETAILED_data.append(l)
                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                        #print(eachfolder+eachfile , mswordtype ,  eachinstance ,  l)
            continue
        
        if (".doc" in eachfile or ".docx" in eachfile):
            brandsString=""
            brandCountDict ,returnDict= _DOC(eachfile,eachfolder,brandings)
            fields = findMailMERGETemplate(eachfile,eachfolder,brandings)
            if (fields > 0):
                mswordtype = "MS WORD-TEMPLATE"
            else:
                mswordtype = ("MS WORD")
            #print("findMailMERGETemplate(eachfile,eachfolder,brandings)",findMailMERGETemplate(eachfile,eachfolder,brandings))
            if (len(brandCountDict) > 0):
                Row_data.clear()
                finalCount=""
                finalBrandString=""
                for eachinstancecount in brandCountDict:	
                    finalBrandString = finalBrandString + eachinstancecount + ";"
                    finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                Row_data.append(eachfolder+eachfile )
                Row_data.append(mswordtype)
                Row_data.append(finalBrandString)
                Row_data.append(finalCount)
                #print(eachfolder+eachfile,brandsString,count)
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            else:
                Row_data.clear()
                Row_data.append(eachfolder+eachfile )
                Row_data.append(mswordtype)
                Row_data.append("NONE")
                Row_data.append("NA")
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            if len(returnDict) > 0 :
                for eachinstance in returnDict:
                    #list = []
                    list = returnDict[eachinstance]
                    #print(":::list",list)
                    for l in list:
                        #print("inside" , l)
                        Row_DETAILED_data.clear()
                        Row_DETAILED_data.append(eachfolder+eachfile )
                        Row_DETAILED_data.append(mswordtype)
                        Row_DETAILED_data.append(eachinstance)
                        Row_DETAILED_data.append(l)
                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                        #print(eachfolder+eachfile , mswordtype ,  eachinstance ,  l)
            continue
        if (".zip" in eachfile or ".jar" in eachfile ):
            import zipfile
            gmt1 = time.gmtime() 
            ts1 = calendar.timegm(gmt1)
            ts1=str(ts1)
            zipfolder=eachfolder+eachfile.replace(".zip",ts1).replace(".jar",ts1) + "ZIP-JAR\\" 
            #print("##########zipfolder",zipfolder)
            
            with zipfile.ZipFile(eachfolder+eachfile, 'r') as zip_ref:
                zip_ref.extractall(zipfolder)
            #zipfiles = [f1 for f1 in listdir(zipfolder) if isfile(join(zipfolder, f1))]
            zipsubdirs = [x[0] for x in os.walk(zipfolder)]
            for eachfolderinzip in zipsubdirs:
                eachfolderinzipfiles = [f2 for f2 in listdir(eachfolderinzip) if isfile(join(eachfolderinzip, f2))]
                for eachzipfile in eachfolderinzipfiles:
                    lFileContents=[]
                    #print("Processing zipfolder or Jar..." + eachfolderinzip+eachzipfile)
                    if ("~$S" in eachzipfile):
                        print("Skipping Lock Files:" + eachfolderinzip+"\\"+eachzipfile )
                        continue
                    if (".html" in eachfile):
                        brandsString=""
                        brandCountDict,returnDict= _HTML(eachzipfile,eachfolderinzip,brandings)
                        if (len(brandCountDict) > 0):
                            Row_data.clear()
                            finalCount=""
                            finalBrandString=""
                            for eachinstancecount in brandCountDict:	
                                finalBrandString = finalBrandString + eachinstancecount + ";"	
                                finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                            Row_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                            Row_data.append("HTML")
                            Row_data.append(finalBrandString)
                            Row_data.append(finalCount)
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        else:
                            Row_data.clear()
                            Row_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                            Row_data.append("HTML")
                            Row_data.append("NONE")
                            Row_data.append("NA")
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                            if len(returnDict) > 0 :
                                for eachinstance in returnDict:
                                    #list = []
                                    list = returnDict[eachinstance]
                                    #print(":::list",list)
                                    for l in list:
                                        #print("inside" , l)
                                        Row_DETAILED_data.clear()
                                        Row_DETAILED_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                                        Row_DETAILED_data.append("HTML")
                                        Row_DETAILED_data.append(eachinstance)
                                        Row_DETAILED_data.append(l)
                                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                                        print(eachfolder+eachfile , "HMTL" ,  eachinstance ,  l)
                        continue
                        
                        
                    if (".xml" in eachfile):
                        brandsString=""
                        brandCountDict,returnDict = _XML(eachzipfile,eachfolderinzip,brandings)
                        if (len(brandCountDict) > 0):
                            Row_data.clear()
                            finalCount=""
                            finalBrandString=""
                            for eachinstancecount in brandCountDict:	
                                finalBrandString = finalBrandString + eachinstancecount + ";"	
                                finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                            Row_data.append(eachfolder+eachfile+"\\"+eachzipfile )
                            Row_data.append("XML")
                            Row_data.append(finalBrandString)
                            Row_data.append(finalCount)
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        else:
                            Row_data.clear()
                            Row_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                            Row_data.append("XML")
                            Row_data.append("NONE")
                            Row_data.append("NA")
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                            if len(returnDict) > 0 :
                                for eachinstance in returnDict:
                                    #list = []
                                    list = returnDict[eachinstance]
                                    #print(":::list",list)
                                    for l in list:
                                        #print("inside" , l)
                                        Row_DETAILED_data.clear()
                                        Row_DETAILED_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                                        Row_DETAILED_data.append("XML")
                                        Row_DETAILED_data.append(eachinstance)
                                        Row_DETAILED_data.append(l)
                                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                                        print(eachfolder+eachfile , "XML" ,  eachinstance ,  l)
                        continue
                    if (".rdf" in eachfile):
                        #continue
                        brandsString=""
                        brandCountDict,returnDict= _RDF(eachzipfile,eachfolderinzip,brandings)
                        if (len(brandCountDict) > 0):
                            Row_data.clear()
                            finalCount=""
                            finalBrandString=""
                            for eachinstancecount in brandCountDict:	
                                finalBrandString = finalBrandString + eachinstancecount + ";"	
                                finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                            Row_data.append(eachfolder+eachfile+"\\"+eachzipfile)
                            Row_data.append("RDF")
                            Row_data.append(finalBrandString)
                            Row_data.append(finalCount)
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        else:
                            Row_data.clear()
                            Row_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                            Row_data.append("RDF")
                            Row_data.append("NONE")
                            Row_data.append("NA")
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                            if len(returnDict) > 0 :
                                for eachinstance in returnDict:
                                    #list = []
                                    list = returnDict[eachinstance]
                                    #print(":::list",list)
                                    for l in list:
                                        #print("inside" , l)
                                        Row_DETAILED_data.clear()
                                        Row_DETAILED_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                                        Row_DETAILED_data.append("RDF")
                                        Row_DETAILED_data.append(eachinstance)
                                        Row_DETAILED_data.append(l)
                                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                                        #print(eachfolder+eachfile , "RDF" ,  eachinstance ,  l)
                        continue
                    if (".pdf" in eachzipfile):
                        brandsString=""
                        brandCountDict ,returnDict= _PDF(eachzipfile,eachfolderinzip,brandings)
                        if (len(brandCountDict) > 0):
                            Row_data.clear()
                            finalCount=""
                            finalBrandString=""
                            for eachinstancecount in brandCountDict:
                                finalBrandString = finalBrandString + eachinstancecount + ";"
                                finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                            Row_data.append(eachfolder+eachfile+"\\"+eachzipfile  )
                            Row_data.append("PDF")
                            Row_data.append(finalBrandString)
                            Row_data.append(finalCount)
                            #print(eachfolder+eachfile+eachzipfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        else:
                            Row_data.clear()
                            Row_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                            Row_data.append("PDF")
                            Row_data.append("NONE")
                            Row_data.append("NA")
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        if len(returnDict) > 0 :
                            for eachinstance in returnDict:
                                list = []
                                list = returnDict[eachinstance]
                                for l in list:
                                    Row_DETAILED_data.clear()
                                    Row_DETAILED_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile)
                                    Row_DETAILED_data.append("PDF")
                                    Row_DETAILED_data.append(eachinstance)
                                    Row_DETAILED_data.append(l)
                                    Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                                    #print("::",eachfolder+eachfile , "PDF" ,  eachinstance ,  l)
                        continue
                    if (".xlsx" in eachzipfile or ".xls" in eachzipfile):
                        brandsString=""
                        brandCountDict ,returnDict= _XLS(eachzipfile,eachfolderinzip,brandings)
                        if (len(brandCountDict) > 0):
                            Row_data.clear()
                            Row_data.append(eachfolder+eachfile+"\\"+eachzipfile )
                            finalCount=""
                            finalBrandString=""
                            for eachinstancecount in brandCountDict:	
                                finalBrandString = finalBrandString + eachinstancecount + ";"
                                finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                            Row_data.append("XL-Sheet")
                            Row_data.append(finalBrandString)
                            Row_data.append(finalCount)
                            #print(eachfolder+eachfile+"-"+eachzipfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        else:
                            Row_data.clear()
                            Row_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                            Row_data.append("XL-Sheet")
                            Row_data.append("NONE")
                            Row_data.append("NA")
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        if len(returnDict) > 0 :
                            for eachinstance in returnDict:
                                #list = []
                                list = returnDict[eachinstance]
                                #print(":::list",list)
                                for l in list:
                                    #print("inside" , l)
                                    Row_DETAILED_data.clear()
                                    Row_DETAILED_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile)
                                    Row_DETAILED_data.append("XL-Sheet")
                                    Row_DETAILED_data.append(eachinstance)
                                    Row_DETAILED_data.append(l)
                                    Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                        continue

                    if (".doc" in eachzipfile or ".docx" in eachzipfile):
                        brandsString=""
                        brandCountDict,returnDict= _DOC(eachzipfile,eachfolderinzip,brandings)
                        fields = findMailMERGETemplate(eachfile,eachfolder,brandings)
                        if (fields > 0):
                            mswordtype = "MS WORD-TEMPLATE"
                        else:
                            mswordtype = ("MS WORD")
                        if (len(brandCountDict) > 0):
                            Row_data.clear()
                            finalCount=""
                            finalBrandString=""
                            for eachinstancecount in brandCountDict:
                                finalBrandString = finalBrandString + eachinstancecount + ";"
                                finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                            Row_data.append(eachfolder+eachfile+"\\"+eachzipfile  )
                            Row_data.append(mswordtype)
                            Row_data.append(finalBrandString)
                            Row_data.append(finalCount)
                            #print(eachfolder+eachfile+"-"+eachzipfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        else:
                            Row_data.clear()
                            Row_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile )
                            Row_data.append(mswordtype)
                            Row_data.append("NONE")
                            Row_data.append("NA")
                            #print(eachfolder+eachfile,brandsString,count)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        continue
                        if len(returnDict) > 0 :
                            for eachinstance in returnDict:
                                #list = []
                                list = returnDict[eachinstance]
                                #print(":::list",list)
                                for l in list:
                                    #print("inside" , l)
                                    Row_DETAILED_data.clear()
                                    Row_DETAILED_data.append(eachfolderinzip+eachfile+"\\"+eachzipfile)
                                    Row_DETAILED_data.append("MS-Word")
                                    Row_DETAILED_data.append(eachinstance)
                                    Row_DETAILED_data.append(l)
                                    Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                                    #print(eachfolder+eachfile , "MS-Word" ,  eachinstance ,  l)
                    if (".zip" in eachzipfile ):
                        print("zip inside zip: Can Not process", eachfolder+eachfile+"-"+eachzipfile)
                        continue
                    try:
                        with open(eachfolderinzip+ "\\" + eachzipfile,"r") as INPFILE:
                            lFileContents=INPFILE.read()
                        INPFILE.close()
                        count=0
                        brandsString=""
                        returnDict={}
                        brandCountDict={}
                        for aBarndItem in brandings:
                            find_the_word = re.finditer(aBarndItem, lFileContents)
                            snippetList =[]
                            for match in find_the_word:
                                start=match.start()-50
                                end=match.end()+50
                                if (start  < 0 ):
                                    start = 0
                                if (len(lFileContents) < end):
                                    end = len(lFileContents)
                                finalstring = lFileContents[start:end].replace("\\n", " ")
                                if len(finalstring) > 250:
                                    finalstring = finalstring[0:150]
                                snippetList.append(finalstring)
                            if (len(snippetList) > 0):
                                brandCountDict[aBarndItem] = len(snippetList)
                                returnDict[aBarndItem] = snippetList
                        
                        i=eachzipfile.index(".")
                        end=len(eachzipfile)
                        fileType=eachzipfile[i+1:end]
                                
                                
                        if (len(brandCountDict) > 0):
                            Row_data.clear()
                            finalCount=""
                            finalBrandString=""
                            for eachinstancecount in brandCountDict:	
                                finalBrandString = finalBrandString + eachinstancecount + ";"	
                                finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                            Row_data.append(eachfolder+eachfile+"\\"+ eachzipfile)
                            Row_data.append(fileType)
                            Row_data.append(finalBrandString)
                            Row_data.append(finalCount)
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        else:
                            Row_data.clear()
                            Row_data.append(eachfolder+eachfile+"\\"+ eachzipfile)
                            Row_data.append(fileType)
                            Row_data.append("NONE")
                            Row_data.append("NA")
                            Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                        if len(returnDict) > 0 :
                            for eachinstance in returnDict:
                                #list = []
                                list = returnDict[eachinstance]
                                #print(":::list",list)
                                for l in list:
                                    #print("inside" , l)
                                    Row_DETAILED_data.clear()
                                    Row_DETAILED_data.append(eachfolder+eachfile+"\\"+ eachzipfile )
                                    Row_DETAILED_data.append(fileType)
                                    Row_DETAILED_data.append(eachinstance)
                                    Row_DETAILED_data.append(l)
                                    Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                                    #print(eachfolder+eachfile , fileType ,  eachinstance ,  l)
                    except Exception:
                        print("notprocessed::problem in file"+eachfolderinzip+ "\\" + eachzipfile,file=exceptionFile)
                        print("problem in zip genericfile File" + eachfolder+eachfile+"-"+eachzipfile )
            continue
        try:
            with open(eachfolder+ "\\" + eachfile,"r") as INPFILE:
                lFileContents=INPFILE.read()
            INPFILE.close()
            count=0
            brandsString=""
            returnDict={}
            brandCountDict={}
            for aBarndItem in brandings:
                find_the_word = re.finditer(aBarndItem, lFileContents)
                snippetList =[]
                for match in find_the_word:
                    start=match.start()-50
                    end=match.end()+50
                    if (start  < 0 ):
                        start = 0
                    if (len(lFileContents) < end):
                        end = len(lFileContents)
                    finalstring = lFileContents[start:end].replace("\\n", " ")
                    if len(finalstring) > 250:
                        finalstring = finalstring[0:150]
                    snippetList.append(finalstring)
                if (len(snippetList) > 0):
                    brandCountDict[aBarndItem] = len(snippetList)
                    returnDict[aBarndItem] = snippetList
            i=eachfile.index(".")
            end=len(eachfile)
            fileType=eachfile[i+1:end]
            if (len(brandCountDict) > 0):
                Row_data.clear()
                finalCount=""
                finalBrandString=""
                for eachinstancecount in brandCountDict:	
                    finalBrandString = finalBrandString + eachinstancecount + ";"	
                    finalCount = finalCount + str(brandCountDict[eachinstancecount]) + ";"
                Row_data.append(eachfolder+eachfile )
                Row_data.append(fileType)
                Row_data.append(finalBrandString)
                Row_data.append(finalCount)
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
                #print(eachfolder+eachfile,brandsString,count)
            else:
                Row_data.clear()
                Row_data.append(eachfolder+eachfile )
                Row_data.append(fileType)
                Row_data.append("NONE")
                Row_data.append("NA")
                Hitachi_CODEBRAND_File_writer.writerow(Row_data)
            if len(returnDict) > 0 :
                for eachinstance in returnDict:
                    #list = []
                    list = returnDict[eachinstance]
                    #print(":::list",list)
                    for l in list:
                        #print("inside" , l)
                        Row_DETAILED_data.clear()
                        Row_DETAILED_data.append(eachfolder+eachfile )
                        Row_DETAILED_data.append(fileType)
                        Row_DETAILED_data.append(eachinstance)
                        Row_DETAILED_data.append(l)
                        Hitachi_DETAILED_CODEBRAND_File_writer.writerow(Row_DETAILED_data)
                        #print(eachfolder+eachfile , fileType ,  eachinstance ,  l)
        except Exception:
            print("notprocessed::problem in file"+eachfolder+ "\\" + eachfile,file=exceptionFile)
            print("problem in General File" + eachfile )
Hitachi_CODEBRAND_File.close()
exceptionFile.close()
Hitachi_DETAILED_CODEBRAND_File.close()

print("Total Files Processed:",fileCount )
print("Code Branding Instance Completed:",DETAILED_CODEBRAND_File,CODEBRAND_File,expfile)

Warning problem in DOCS C:\hitachiPOC\hitachicapitalvehiclesolutions\CODEBASE\\ideal.zip "There is no item named '[Content_Types].xml' in the archive"
Warning problem in DOCS C:\hitachiPOC\hitachicapitalvehiclesolutions\CODEBASE\\ideal.zip "There is no item named '[Content_Types].xml' in the archive"
problem in zip genericfile FileC:\hitachiPOC\hitachicapitalvehiclesolutions\CODEBASE\\org.json.jar-CDL.class
problem in zip genericfile FileC:\hitachiPOC\hitachicapitalvehiclesolutions\CODEBASE\\org.json.jar-Cookie.class
problem in zip genericfile FileC:\hitachiPOC\hitachicapitalvehiclesolutions\CODEBASE\\org.json.jar-HTTP.class
problem in zip genericfile FileC:\hitachiPOC\hitachicapitalvehiclesolutions\CODEBASE\\org.json.jar-JSONArray.class
problem in zip genericfile FileC:\hitachiPOC\hitachicapitalvehiclesolutions\CODEBASE\\org.json.jar-JSONML.class
problem in zip genericfile FileC:\hitachiPOC\hitachicapitalvehiclesolutions\CODEBASE\\org.json.jar-JSONObject.class
problem in zip genericfi

In [59]:
__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"
def Extract_Link (item_url):
    response = requests.get(item_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    all_links = [link.get("href") for link in soup("a")]
    ext_links = [link.get("href") for link in soup("a") if "http" in link.get("href")]
    return(all_links,ext_links)

In [56]:
__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"
def recursiveUrl(url, link, depth):
    if depth == 5:
        return url
    else:
        #print(link['href'])
        page = requests.get(link)
        soup = BeautifulSoup(page.text, 'html.parser')
        newlink = soup.find('a')
        if len(newlink) == 0:
            return link
        else:
            return link, recursiveUrl(url, newlink, depth + 1)

In [57]:
__version__ = "0.2"
__copyright__ = "CopyRight (C) TCSL Kannan"
__license__ = "TCSL"
__author__ = "Kannan Malaichamy"
__author_email__ = "kannan.malaichamy@tcs.com"
def getLinks(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    links = [link.get("href") for link in soup("a") if "http" in link.get("href")]   
    for link in links:
        links.append(recursiveUrl(url, link, 0))
    return links

In [51]:
# links = getLinks("http://www.hitachicapital.co.uk/")
# print(links)

In [ ]:
# item_url = "http://www.hitachicapital.co.uk/"

# #test = soup.findAll(text = re.compile('HCC'))
# #test1 = soup.findAll(text = re.compile('Hitachi Capital Corporation'))
# all_links,ext_links = Extract_Link(item_url)
# print("--------all_links-----")
# print(all_links)
# print("-----ext_links--------")
# print(len(ext_links))

In [ ]:
# Create html from URLs
# for /F %i in (urls.txt) do python d:\shriram\python\P5.py %i

import certifi
import urllib3



import requests
import re
import string
import sys
import os
import csv
def formatHTMLName(fStr):
	fStr=re.sub("[./=?:]","_",fStr)
	return fStr

def createHTMLFromURL(gURL):
    #fURL=re.sub('^(.*)\/\/(.*)$',r'\2',gURL)
    fURL=formatHTMLName(gURL)
    fURL=fURL.replace("/","_")
    output_path = "C:\\hitachiPOC\\hitachipersonalfinance\\RESULT\\"
    Hitachi_gURL_fURL_Mapping_File = open(output_path + 'hitachipersonalfinance_gURL_fURL_Run#1.csv', 'a', newline='')
    Hitachi_gURL_fURL_Mapping_File_writer = csv.writer(Hitachi_gURL_fURL_Mapping_File)
    #Hitachi_gURL_fURL_Mapping_File_File_writer.writerow(["gURL","fURL"])
    Row_data = []
    Row_data.append(gURL)
    Row_data.append(fURL )
    Hitachi_gURL_fURL_Mapping_File_writer.writerow(Row_data)
    print(gURL+":"+fURL)
    fOutFileName=OUTDIR+"HTML\\"+fURL+".html"
    Hitachi_gURL_fURL_Mapping_File.close() 
    if not os.path.exists(fOutFileName):
        try:
            r = requests.get(gURL, allow_redirects=True,verify=False)
        except Exception:
            print("Cannot Open URL:[%s]" % (gURL))
            return 90
        #print("fOutFileName:"+fOutFileName+":")
        try:
            with open(fOutFileName,"w") as outFile:
                outFile.write(str(r.content))
        except Exception:
            print("Cannot Open File:[%s]" % (fOutFileName))
            return 80
        print("Created File:%s:" % (fOutFileName))
        OUTFILE.write(fOutFileName+"\n")
        return 0
    else:
        print("File exists:%s:. Skipping..." % (fOutFileName))
    #		print("\n%s" % (fOutFileName),file=OUTFILE,end="")
        OUTFILE.write(fOutFileName+"\n")
    return 10


#DEBUG=os.getenv('DEBUG')
#DEBUG=int(DEBUG)
OUTDIR='c:\\hitachiPOC\\hitachipersonalfinance\\'

if __name__ == "__main__":
	
	with open(OUTDIR+"hitachipersonalfinance-urls.txt","r") as URLFILE:
		try:
			OUTFILE=open(OUTDIR+"hitachipersonalfinance-htmls.txt","w")
		except Exception:
			print("CANNOT OPEN OUTFILE:%s:" % (OUTDIR+"hitachipersonalfinance-htmls.txt"))

		for aFile in URLFILE:
			aFile=str(aFile)
			aFile=re.sub('\\n|\\r','',aFile)
			createHTMLFromURL(aFile)
	OUTFILE.close()

In [11]:
# Try Authentication Pages 
import requests

response = requests.get('https://www.hitachicapitalvehiclesolutions.co.uk/',auth=(
                            'kannan.malaichamy@gmail.com', 'tcs50@HARI'))
print(response.text)

{"message": "Client errors"}


In [10]:
request = urllib.request.Request("https://www.hitachicapitalvehiclesolutions.co.uk/")
handle = urllib.request.build_opener()
data=handle.open(request)
mime_type=data.info().get_content_type()
url=data.geturl();
print(url)
content = data.read().decode("utf-8", errors="replace")
print(content)

HTTPError: HTTP Error 403: Forbidden

In [9]:
import urllib.parse
import urllib.request
import urllib.response

emailAddress = "kannan.malaichamy@gmail.com"
password  = "tcs50@HARI"
top_level_url = "https://www.hitachicapitalvehiclesolutions.co.uk/find-a-vehicle/car/results"

# create an authorization handler
p = urllib.request.HTTPPasswordMgrWithDefaultRealm()
p.add_password(None, top_level_url, username, password)

auth_handler = urllib.request.HTTPBasicAuthHandler(p)

opener = urllib.request.build_opener(auth_handler)

urllib.request.install_opener(opener)

try:
    result = opener.open(top_level_url)
    messages = result.read()
    print (messages)
except IOError as e:
    print (e)

HTTP Error 403: Forbidden


In [5]:
import urllib
import urllib2
import http.cookiejar
cookielib = http.cookiejar

username = 'kannan.malaichamy@gmail.com'
password = 'tcs50@HARI'

cj = cookielib.CookieJar()
opener = urllib.build_opener(urllib.HTTPCookieProcessor(cj))
login_data = urllib.urlencode({'username' : username, 'j_password' : password})
opener.open('https://www.hitachicapitalvehiclesolutions.co.uk/', login_data)
resp = opener.open('https://www.hitachicapitalvehiclesolutions.co.uk/find-a-vehicle/car/results')
print (resp.read()) 

ModuleNotFoundError: No module named 'urllib2'

In [16]:
from requests import session

payload = {
    'action': 'login',
    'emailAddress': "kannan.malaichamy@gmail.com",
    'password': "tcs50@HARI"
}

with session() as c:
    c.post('https://www.hitachicapitalvehiclesolutions.co.uk/', data=payload)
    response = c.get('https://www.hitachicapitalvehiclesolutions.co.uk/account')
    print(response.headers)
    print(response.text)
    print(response)

{'Date': 'Wed, 10 Feb 2021 12:34:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '28', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains;', 'Referrer-Policy': 'no-referrer-when-downgrade', 'Content-Security-Policy': "default-src * data: 'unsafe-eval' 'unsafe-inline'"}
{"message": "Client errors"}
<Response [403]>
